In [5]:
import datetime
import rasterio
import pandas as pd

In [6]:
def get_pixel_boundary_wkt(dataset, row, col):
    points = [
        dataset.xy(row, col, offset=offset) for offset in ["ul", "ll", "lr", "ur", "ul"]
    ]

    return "POLYGON ((" + ", ".join(f"{x} {y}" for x, y in points) + "))"

In [7]:
# src = rasterio.open("../planet/tmp/data/CANOPY_COVER_v1.2.0_30-20190101T000000Z_cc.tiff")
src = rasterio.open("../planet/tmp/data/CANOPY_COVER_v1.0.0_3-20230621T000000Z_cc-uc-reprojection_000025.tiff")

In [8]:
crs = f"EPSG:{src.crs.to_epsg()}"
created_at = datetime.datetime.now(datetime.timezone.utc).isoformat()
data = src.read()

In [12]:
def get_results(start_position, max_position):
    count = -1
    results = []
    for raster_row in range(0, src.height):
        for raster_col in range(0, src.width):
            x, y = src.xy(raster_row, raster_col)
            for raster_band in range(0, src.count):
                count += 1
                if count < start_position:
                    continue
                if count >= max_position:
                    return pd.DataFrame(results)
                results.append({
                    "relative_path": "relative_path",
                    "organisation_id": "organisation_id",
                    "aoi_id": "aoi_id",
                    "data_request_id": "data_request_id",
                    "crs": crs,
                    "x": x,
                    "y": y,
                    "raster_band": raster_band+1,
                    "boundary": get_pixel_boundary_wkt(src, raster_row, raster_col),
                    "value": data[raster_band][raster_row][raster_col],
                    "dataset_id": "dataset_id",
                    "created_at": created_at,
                })
    return pd.DataFrame(results)

In [ ]:
i = -1
while True:
    i += 1

    start_position = 1_000_000 * i
    end_position = 1_000_000 * i + 1_000_000
    print(start_position, end_position)
 
    results = get_results(start_position, end_position)
    if len(results) == 0:
        break

    results.to_parquet(f"../planet/tmp/data/performance_test/1M_{i}.parquet")

0 1000000
1000000 2000000


In [16]:
import pandas as pd

In [21]:
df = pd.read_parquet("/tmp/output.parquet")

In [22]:
df.head()

,rasterBand,rasterCol,rasterRow,value
0,1.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0
2,1.0,2.0,0.0,0.0
3,1.0,3.0,0.0,6.0
4,1.0,4.0,0.0,13.0


In [18]:
src = rasterio.open("/Users/brian/dev/cecilearth/planet/tmp/data/CANOPY_COVER_v1.2.0_30-20190101T000000Z_cc.tiff")

In [19]:
src.shape

(37, 45)

In [20]:
37*45

1665

In [23]:
rasterio.__version__

'1.4.3'

In [28]:
src.meta["crs"].to_epsg()

4326

In [30]:
metadata=src.meta.copy()

In [32]:
metadata["crs"] = src.meta["crs"].to_epsg()

In [34]:
import json
json.dumps(metadata)

'{"driver": "GTiff", "dtype": "uint8", "nodata": 255.0, "width": 45, "height": 37, "count": 1, "crs": 4326, "transform": [0.00025, 0.0, 132.52925, 0.0, -0.00025, -12.721, 0.0, 0.0, 1.0]}'

In [35]:
import pandas as pd

In [36]:
pd.__version__

'2.3.0'

In [37]:
import pyarrow as pa

In [38]:
pa.__version__

'20.0.0'